[120]Magna Graecia\
Sergio Enrico Pisoni 132855\
Sofia Papaioannou 132898\
Lefteris Verouchis 132873

# REPORT

## Exploratory data Analysis (terry)

!!!

Search domain knowledge

Check if the data is intuitive


Understand how the data was generated

Explore individual features

Explore pairs and groups of features

Clean up features

## Models experimentation

### RANDOM FOREST

### XGBOOST

## Feature Engineering

## Model Interpretation